#0. Dependencies

In [2]:
import math
import pandas as pd

In [3]:
# cadCAD configuration modules
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment

# cadCAD simulation engine modules
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor

#1. STATE VARIABLES
A state variable is one of the set of variables that are used to describe the mathematical
"state" of a dynamical system.

In [4]:
initial_state = {'population':50, 'food':1000 }
initial_state

{'population': 50, 'food': 1000}

#2. SYSTEM PARAMETERS
It is the process of choosing parameters that impact the behaviour of the
model. These parameters allow us to perform simulation techniques like
parameter sweeps, Monte Carlo simulations, A/B tests, and see how the system
behaves under a different model paramter set.

In [5]:
system_params = { 'reproduction_rate': 0.01, 'consumption_rate':0.01 }
system_params

{'reproduction_rate': 0.01, 'consumption_rate': 0.01}

#3. POLICY FUNCTIONS
A policy function computes one or more signals to be passed to State Update
Functions. They describe the logic and behaviour of a system component or
mechanism.

#4. STATE UPDATE FUNCTIONS
We create State Update Functions to design the way our model state changes
over time. These will usually represent the system differential specification.

Technically, state update functions are regular python functions that take
5 arguments as input and then return a python tuple. Eq -

def state_update_function(params,substep,state_history, previous_state, policy_input):
    variable_value = 0
    return 'variable_name', variable_value

--> Params : A python dictionary containing system parameters.
--> substep : An integer value representing a step within a single timestep
--> state_history : A python list of all previous states
--> previous_state : A python dict. that defines what the state of the system
    was at the previous timestep.
--> policy_input : A python dict. of signals / actions from policy functions.

In [6]:
def new_population(_current_population, _alpha, _food_supply) :
    """
    The population state after 1 timestep, according to the differential equation (1):
    current_population + alpha * food_supply
    """
    return math.ceil(_current_population + _alpha * _food_supply)

In [7]:
# Relevant state variables
current_population = initial_state['population']
food_supply = initial_state['food']

#Relevant parameters
reproduction_rate = system_params['reproduction_rate']

new_population(current_population,reproduction_rate,food_supply)

60

In [8]:
def s_population(params, substep, state_history, previous_state, policy_input):
    """
    Update the population state according to the differential equation (1):
    current_population + alpha * food_supply
    """
    population = previous_state['population']
    alpha = params['reproduction_rate']
    food_supply = previous_state['food']

    return 'population',max(new_population(population,alpha,food_supply),0)

In [9]:
def s_food(params, substep, state_history, previous_state, policy_input):
    """
    Update the food supply state according to the differential equation (2):
    food_supply - beta * population
    """
    food = previous_state['food'] - params['consumption_rate'] * previous_state['population']
    return 'food',max(food,0)

#5. PARTIAL STATE UPDATE BLOCKS
Tying it All Together : A Series of Partial State Update Blocks is a structure for composing State Update Functions and
policy functions in series or parallel, as a representation of the system model.
In one Time-step, multiple state updates could occur. These updates can either be run in series, or parallel at the
same-time.
So a set of partial state update blocks represents the structure of the entire model, and all updates are run during
a single time-step.
A single partial state update block represents 1 sub-step. Together they make 1 time-step.

So, a python data structure to represent a set of 2 partial state update blocks is a list of 2 python dictionaries.
State update functions in a single block have access to the same previous state, run at the same time.

In [10]:
# UPDATES RUN IN SERIES
partial_state_update_blocks = [
    #Run first
    {
        'policies': {},
        # State variables
        'variables': {
            'population': s_population
        }
    },
    #Run Second
    {
        'policies': {},
        # State variables
        'variables': {
            'food': s_food
        }
    }
]

In [11]:
# UPDATES RUN IN PARALLEL
partial_state_update_blocks = [
    {
        'policies': {},
        # State variables
        'variables': {
            'population': s_population,
            'food': s_food
        }
    }
]

#6. CONFIGURATION
The configuration state is about tying all the previous model components together and choosing how the simulation should run.
MODELLING : State Variables --> System Parameters --> Policy Functions --> State Update Functions --> Partial State Update Blocks
SIMULATION : CONFIGURATION --> EXECUTION --> OUTPUT PREPARATION --> ANALYSIS

cadCAD allows everyone to configure a number of simulation configuration parameters. We have :
--> 'N', which is the number of times we will run the simulation, called "Monte Carlo runs", when we will later look
at tools to analyze system models
--> 'T', which is the number of times the simulation will run for
--> 'M', which is M, the parameters of the system


In [12]:
# config_sim function was imported earlier. It informs cadCAD about the scope of the simulation.
sim_config = config_sim({
    "N" : 1,
    "T" : range(400),
    "M" : system_params
})

TypeError: object of type 'float' has no len()

In [27]:
from cadCAD import configs
#cadCAD stores simulation configuration in a variable called configs, which is a list. So we use del configs[:]
# to delete any / all configurations before we continue.
del configs[:] # Clear any prior configs

In [28]:
# here we are adding initial state, partial state update blocks and sim config to the cadCAD Context.
experiment = Experiment()
# so each cadCAD experiment has configuration options which we will pass to the append configs function to create
# our simulation configuration
experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks= partial_state_update_blocks,
    sim_configs = sim_config
)
# The config object is what the cadCAD Executor uses to understand what to run.
configs[-1].dict_

NameError: name 'sim_config' is not defined

#7. EXECUTION
cadCAD has an engine that runs the simulations. This is the core of cadCAD. It takes our model and configuration
and gives us our simulation results after 'T' time-steps.
So we create an ExecutionContext, and the ExecutionContext decides how the cadCAD should run the simulation. The
default automatically selects between single-threaded or multi-threaded modes, to optimize the performance of the
simulation, or how fast it runs!

Configuring the cadCAD simulation execution

In [13]:
exec_mode = ExecutionMode()
exec_context = ExecutionContext()

In [14]:
# using the cadCAD Executor, the cadCAD simulation Context and the model configuration that we created earlier,
# we can now create a simulation Object here.
simulation = Executor(exec_context, configs = configs)

NameError: name 'configs' is not defined

Time to Simulate our ecosystem model!

In [ ]:
# To run the simulation, we run the execute() function of the simulation object.
raw_result, tensor_field, sessions = simulation.execute()

#8. SIMULATION OUTPUT PREPARATION
The simulation results are returned as a Pandas data-frame. At this stage of the process you will manipulate and
analyze your results to answer questions about your model.

In [ ]:
# we are converting raw_result to a Pandas Dataframe, which allows us to easily analyze rows and columns (just like
# a spreadsheet)
simulation_result = pd.DataFrame(raw_result)

#9. SIMULATION ANALYSIS
The purpose of simulation analysis is to make sure that the output data makes sense, and to answer the original questions
that were asked during the system requirements phase.

In [ ]:
#with this, we will be able to use plotly directly with pandas by calling the plot method on our Pandas dataframe.
pd.options.plotting.backend = 'plotly'

In [ ]:
simulation_result.plot(kind='line',x='timestep',y=['population','food'])

In [ ]:
pd.set_option('display.max_rows',len(simulation_result))
display(simulation_result)

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html
simulation_result.query('food == 0').head()

#SYSTEM VALIDATION PHASE
--> Computational experiments (What-if questions)
POLICY FUNCTIONS are another core feature of cadCAD.
We can use Policy functions to re-factor and improve our model.

POLICY FUNCTION : Policy Function describes the logic of a component of a system / or an input to the system, 
where that input is from outside 